# Deep Learning in Bioimaging Workshop

> Author: Prateek Verma  
> Notebook created for Data Science Core Workshop Series, AIMRC @ U Arkansas  
> This Notebook is designed to run in Google Colab.

Welcome to the Deep Learning in Bioimaging Workshop! In this workshop, we will explore various deep learning techniques and apply them to biomedical images using Fiji (ImageJ) and Google Colab. This workshop is designed to be executed by beginners, so don't worry if you are new to deep learning or bioimaging. Even if you don't understand the code, you will get familiar with the concepts and the workflow for applying state-of-the art deep learning techniques to bioimaging.

We will cover the following exercises:
1. Image Classification (CNN, Google Colab)
2. Image Segmentation (Fiji)
3. Image Segmentation (U-Net, Google Colab)
4. Image Denoising (ZeroCostDL4Mic, Google Colab)

## Pre-workshop preparation

Steps to upload denoising dataset to your Google Drive.
1. Click [this link](https://zenodo.org/records/5750174/files/Denoising_Dataset.zip?download=1) to download the Denoising dataset to your local computer.
2. Extract the downloaded zip file to your local computer.
3. In your Google Drive page in your browser, go to `+ New > Folder upload` and upload the extracted `Denoising_Dataset` folder.

## Exercise 1: Image Classification (Google Colab)

In this exercise, we will build a simple image classification model using a neural network to classify biomedical images into different categories. We will use the MedMNIST dataset for this purpose.

**NOTE:** In the menubar, go to `Runtime > Change runtime type` and select `T4 GPU` for the **entirety of this workshop**.

In [ ]:
# import some common libraries
import numpy as np
import matplotlib.pyplot as plt

### 1. Download the dataset

The first step is to download the dataset. Here, we will download the pathmnist subset of the MedMNIST dataset and save it to disk.

In [ ]:
# define the url
pathmnist_url = "https://zenodo.org/records/10519652/files/pathmnist.npz?download=1"

# import necessary libraries
import os, urllib

# download file from a url
if not os.path.exists("pathmnist.npz"):
    urllib.request.urlretrieve(pathmnist_url, "pathmnist.npz")

### 2. Load the dataset in to memory

Now that the dataset is available, we will load it into memory as train, test, and validation sets.

In [ ]:
# Load the downloaded dataset
data = np.load("pathmnist.npz")
print("Keys available in the data:\n", data.files)

# split data into train, validation, and test sets and corresponding labels
train_images, train_labels = data["train_images"], data["train_labels"]
val_images, val_labels = data["val_images"], data["val_labels"]
test_images, test_labels = data["test_images"], data["test_labels"]

# Normalize the images to have pixel values between 0 and 1
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

print("Shapes of the data:")
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Val images shape:", val_images.shape)
print("Val labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

# also see how many classes are there in the dataset
print("Number of classes in the dataset:", len(np.unique(train_labels)))

### 3. Visualize the dataset

In [ ]:
# Let's plot some of the images along with their labels
fig, axes = plt.subplots(1, 10, figsize=(10, 3))
for i, ax in enumerate(axes):
    ax.imshow(train_images[i])
    ax.set_title(train_labels[i])
    ax.axis("off")
plt.show()

### 4. Prepare data for training

For training, data needs to be fed into the neural network in batches. The process shown here is for Tensorlow/Keras.

In [ ]:
# import tensorflow libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# convert the labels to one-hot encoded format
train_labels = to_categorical(train_labels)
val_labels = to_categorical(val_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Create a data generator object that will be later used to generate batches of (optionally, augmented) images 
train_datagen = ImageDataGenerator() # No augmentation
val_datagen = ImageDataGenerator() # No augmentation
test_datagen = ImageDataGenerator() # No augmentation

train_generator = train_datagen.flow(train_images, train_labels, batch_size=128)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=128)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=128)

### 5. Build a neural network model for classification

We will build a simple neural network model using Keras for image classification.

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)), 
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(9, activation='softmax')
])

# compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# print the model summary
model.summary()

### 6. Train the model

You can train for 20-50 epochs on a GPU inside of 5 minutes. Training for 5 epochs will take about 5-10 minutes when using a CPU kernel.

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Visualize the training process.

In [ ]:
# Let's visualize the training and validation accuracy and loss over the epochs.
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

### 7. Evaluate the results

We will plot a confusion matrix for the test dataset and calculate the accuracy of the model.

In [ ]:
# import necessary libraries
from sklearn.metrics import confusion_matrix
import seaborn as sns

# predict the test labels
test_predictions = model.predict(test_images)

# get the predicted labels
test_predictions = np.argmax(test_predictions, axis=1)

# get the true labels
test_true = np.argmax(test_labels, axis=1)

# print the accuracy
print("Accuracy on test set:", np.mean(test_predictions == test_true))

In [ ]:
# calculate the confusion matrix
cm = confusion_matrix(test_true, test_predictions)

# plot the confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()

## Exercise 2: Using Fiji and DeepImageJ

In this exercise, we will learn how to use Fiji (ImageJ) and deep learning models using plugins like `DeepImageJ` and model repositories such as `Bioimage.io`.

Please follow the **instructions in the Workshop slides** to complete this exercise.

You'll learn:
- How to download and run Fiji (ImageJ)
- How to install and use DeepImageJ plugin
- How to use Bioimage.io model repository to download pre-trained deep learning models
- How to run deep learning models to your images

**Author's note** (read after the workshop): There are many pros and cons of using Fiji and plugins such as DeepImageJ. This is a quick way to get started with deep learning in bioimaging without writing any code. This is also a good way to stay up-to-date on popular tasks and models in the BioImaging community. However, you will soon realize that many of these models become outdated and incompatible (because of various software and environment that end users use) quickly and might be overly-complicated for your need because they try to generalize to a larger audience. The last point is especially true for open source deep-learning tools such as ZeroCostDL4Mic (covered later in the workshop). As you will learn in this workshop, it is possible to write simple, clean and effective Python code for your own project using libraries like TensorFlow, PyTorch, etc. to carry out similar tasks.

## Exercise 3: Image Segmentation with U-Net (Google Colab)

In this exercise, we will build a U-Net model for image segmentation using the Kvasir-SEG dataset, which contains gastrointestinal polyp images and their corresponding segmentation masks.

### Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import cv2
import urllib
import zipfile

### Download and extract the dataset

We will use the Kvasir-SEG dataset for this exercise. Let's download and extract the dataset.

In [ ]:
# Download the Kvasir-SEG dataset to local storage
url = "https://datasets.simula.no/downloads/kvasir-seg.zip"
zip_path = "Kvasir-SEG.zip"
data_path = ""

if not os.path.exists(zip_path):
    urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_path)

### Load the dataset

Let's load the images and masks from the extracted dataset.

In [ ]:
# define paths for images and masks
image_dir = os.path.join(data_path, 'Kvasir-SEG/images')
mask_dir = os.path.join(data_path, 'Kvasir-SEG/masks')

# create empty lists to store images and masks
images = []
masks = []

# loop through each file in the images and masks directories, read the images and masks, and store them in the lists
for image_name in os.listdir(image_dir):
    
    image_path = os.path.join(image_dir, image_name)
    mask_path = os.path.join(mask_dir, image_name)

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    image = cv2.resize(image, (64, 64))
    mask = cv2.resize(mask, (64, 64))

    images.append(image)
    masks.append(mask)

# convert the lists to numpy arrays
images = np.array(images)
masks = np.array(masks)

# Normalize images
images = images / 255.0
masks = masks / 255.0
masks = masks.reshape(-1, 64, 64, 1) # add an extra dimension for the channel

# Split the data into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

# Check the shapes to ensure they are correct
print(f'Train images shape: {train_images.shape}')
print(f'Validation images shape: {val_images.shape}')
print(f'Train masks shape: {train_masks.shape}')
print(f'Validation masks shape: {val_masks.shape}')

### Build the U-Net model

We will build a U-Net model for image segmentation.

In [ ]:
def unet_model(input_size=(64, 64, 3)):
    
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)

    # Decoder
    up5 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop4))
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(merge5)
    conv5 = Conv2D(128, 3, activation='relu', padding='same')(conv5)

    up6 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv5))
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(64, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(32, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(32, 3, activation='relu', padding='same')(conv7)
    
    conv8 = Conv2D(1, 1, activation='sigmoid')(conv7)

    model = Model(inputs=inputs, outputs=conv8)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = unet_model()
model.summary()

### Train the model

Now, we will train the model using the training data. Training for 25 epochs would have taken about an hour on a CPU kernel, hence we are using a **GPU kernel**. On a GPU kernel, this takes about a minute.

In [ ]:
history = model.fit(
    train_images, train_masks,
    epochs=25,
    batch_size=32,
    validation_data=(val_images, val_masks)
)

### Evaluate the results

Finally, we will evaluate the model's performance on the validation data.

In [ ]:
plt.figure(figsize=(9, 3))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

#### Visualize segmentation results

Let's visualize some example segmentation results.

In [ ]:
def display_results(model, images, labels, num_samples=10):
    predictions = model.predict(images)
    fig, axes = plt.subplots(num_samples, 3, figsize=(6, 18))

    for i in range(num_samples):
        axes[i, 0].imshow(images[i].reshape(64, 64, 3))
        axes[i, 0].set_title('Input Image')
        axes[i, 1].imshow(labels[i].reshape(64, 64), cmap='gray')
        axes[i, 1].set_title('True Mask')
        axes[i, 2].imshow(predictions[i].reshape(64, 64), cmap='gray')
        axes[i, 2].set_title('Predicted Mask')
        for ax in axes[i]:
            ax.axis('off')

    plt.show()

display_results(model, val_images, val_masks)

## [Optional] Exercise 4: Denoising using CARE (via ZeroCostDL4Mic)

**NOTE:** You should have the Denoising_Dataset folder already uploaded to your Google Drive (see pre-workshop preparation section at the top).

In this exercise, we will use the CARE (Content-Aware Image Restoration) neural network to denoise biomedical images. We will follow the Notebook implementation of ZeroCostDL4Mic directly, a platform that provides free access to deep learning models for bioimaging.

1. Please click [here](https://colab.research.google.com/github/HenriquesLab/ZeroCostDL4Mic/blob/master/Colab_notebooks/CARE_2D_ZeroCostDL4Mic.ipynb) to open the ZeroCostDL4Mic notebook in a new Google Colab window.
2. Delete the current Google Colab session.  
Go to `Runtime > Manage Sessions` and delete the current session.  
This is because you can only run one notebook at a time on the free tier.
3. Create a copy of the ZeroCostDL4Mic notebook.  
`File > Save a copy in Drive`.
4. Connect to the `T4 GPU` runtime in the ZeroCostDL4Mic notebook. Not in this notebook. Training for 50 epochs takes about 2 minutes on a GPU kernel.
5. Follow the instructions in the ZeroCostDL4Mic notebook. Just run all the cells one by one starting from Section 1.1. You'll need to change a few things along the way as mentioned below.

### Things to change/enter in the notebook:

You might need to adjust some of these based on where you uploaded the dataset in your Google Drive. Leave all other settings as they are in the notebook.

- **Section 3.1**  
Training_source: `/content/gdrive/MyDrive/Denoising_Dataset/Training/low`  
Training_target: `/content/gdrive/MyDrive/Denoising_Dataset/Training/high`  
model_name: `denoising_model`  
model_path: `/content/gdrive/MyDrive/Denoising_Dataset/Results`  
number_of_epochs: `5` you can increase/decrease this number depending on whether or not you are connected to a GPU kernel and/or if you have time`  

- **Section 4.2** (Important)  
Change lr to learning_rate in this line  
`writer.writerow([history.history['loss'][i], history.history['val_loss'][i], history.history['learning_rate'][i]])`  

- **Section 5.2**  
Source_QC_folder: `/content/gdrive/MyDrive/Denoising_Dataset/test/Low`  
Target_QC_folder: `/content/gdrive/MyDrive/Denoising_Dataset/test/High`  

- **Section 6.1**  
Data_folder: `/content/gdrive/MyDrive/Denoising_Dataset/test/Low`  
Result_folder: `/content/gdrive/MyDrive/Denoising_Dataset/Results`